### миниспич зачем все это вообще нужно

### Как это вообще обычно выглядит под капотом<br>

<img src='../img/auth_flow.png' alt="loshara, net faila" height="500" width="500">

## Коннектимся к Reporting API v4

### Устанавливаем пакеты

In [3]:
!pip install --upgrade google-api-python-client
!pip install --upgrade oauth2client

Requirement already up-to-date: google-api-python-client in /Users/bacchus/miniconda3/lib/python3.6/site-packages (1.7.7)
Requirement already up-to-date: oauth2client in /Users/bacchus/miniconda3/lib/python3.6/site-packages (4.1.3)


### Создаем сервисный аккаунт

<p>Для справки можно использовать вот эту ссылку https://developers.google.com/analytics/devguides/reporting/core/v4/authorization#service_accounts </p>

Во всем нижеследующем тексте я предполагаю что вы уже создали себе аккаунт в google cloud, так как у нас уже был урок связанный с базами данных где мы это делали.

<ul>
    <li>Заходим на cloud.google.com</li>
    <li>Далее в раздел IAM & Admin > Service accounts <img src="../img/serv_acc.png" alt="loshara net faila" height="350" width="350"></li>
    <li>Создаем в нем сервисный аккаунт</li>
    <li>Создаем json key</li>
    <li>Делаем доступной нашему проекту Analytics API на вкладке APIs & Services</li>
    <li>Даем разрешения на чтение имейлу сервисного аккаунта в ту view из которой хотим забирать данные</li>
</ul>

Чем отличается сервисный аккаунт? Он никак не привязан к персональным данным пользователей, поэтому взаимодействоавть через него с сервисами
гугла гораздо проще, как пример можно посмотреть разные схемы авторизации: <br>
https://developers.google.com/identity/protocols/OAuth2 <br>
Он изначально принадлежит машине, и предназначен для взаимодействий по типу server-server. Нам именно это и нужно, ведь мы по итогу хотим нашими приложениями забрать много данных под разными срезами из гугл аналитикс или по расписанию обновлять данные за периоды. <br>
В случаях авторизации НЕ через сервисный аккаунт требуется авторизация через пользователя.


Упрощенно схема авторизации через сервисный аккаунт выглядит вот так: <img src='../img/auth_schema.png' alt="loshara net faila" height="350" width="350">

### Авторизовываемся

In [4]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

KEY_FILE_LOCATION = 'path'
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

In [5]:
# получаем токен
# справка по объекту https://oauth2client.readthedocs.io/en/latest/source/oauth2client.service_account.html
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

In [6]:
type(credentials)

oauth2client.service_account.ServiceAccountCredentials

In [ ]:
# credentials.__dict__

In [7]:
# создаем объект для взаимодействия с апишкой
# https://googleapis.github.io/google-api-python-client/docs/epy/googleapiclient.discovery-module.html#build
analytics = build('analyticsreporting', 'v4', credentials=credentials)

In [8]:
type(analytics)

googleapiclient.discovery.Resource

In [ ]:
# analytics.__dict__

####  Оформим все эти пироги в одну функцию чтобы она возвращала уже готовый объект

In [9]:
def get_credentials(KEY_FILE_LOCATION):
    """Initializes an Analytics Reporting API V4 service object.
    Returns:
      An authorized Analytics Reporting API V4 service object.
    """
    # get token
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

    # build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

In [11]:
test = get_credentials(KEY_FILE_LOCATION)

In [12]:
type(test)

googleapiclient.discovery.Resource

## Ну и естественно теперь все это перенесем в отдельнуый модуль чтобы не таскать каждый раз эти громоздкие конструкции

Переносим все что носит константный характер, расположение ключа к ним не относится, поэтому его не переносим.